In [1]:
import os
# We save the model to a local mounted directory
os.environ['HF_HOME'] = '/workspace/llm_models'

modality = "passage"

In [2]:
import sys
sys.path.append('/workspace/src/')

In [3]:
from dotenv import dotenv_values, load_dotenv
from umbrela.gpt_judge import GPTJudge
from umbrela.osllm_judge import OSLLMJudge 
import torch
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

import pickle

import json
from tqdm import tqdm

from preprocessing.utils import parallel_process_topics, single_process_topics, load_vectorstore, load_pool_documents

db_vals = dotenv_values("/workspace/src/.env")
load_dotenv()

True

In [4]:
passage_pool = json.load(open("/workspace/src/data/passage_pool.json", "r"))
table_pool = json.load(open("/workspace/src/data/table_pool.json", "r"))

passage_pool_for_rel = json.load(open("/workspace/src/data/passage_pool_for_rel.json", "r"))

topics = pickle.load(open("/workspace/src/data/topics.pkl", "rb"))
vectorstore = load_vectorstore("/workspace/src/preprocessing/vectorstores/chromadb_store")

/workspace/src/preprocessing/utils.py:87: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(model=embedding_model)
/workspace/src/preprocessing/utils.py:88: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [18]:
if modality == 'table':
    pool_documents = load_pool_documents(table_pool, vectorstore)
else:
    pool_documents = load_pool_documents(passage_pool, vectorstore)

In [26]:
model_names = ["mistralai/Mistral-Small-Instruct-2409",
    "microsoft/phi-4",
    "mistralai/Mistral-7B-Instruct-v0.3",
    "tiiuae/Falcon3-7B-Instruct",
               
               "meta-llama/Llama-3.2-3B-Instruct",  
               "Qwen/Qwen2.5-14B-Instruct", 
                
               "google/gemma-2-9b-it",]

In [ ]:
#gauge execution time
start_time = time.time()    

for model_name in model_names:

    with torch.cuda.device(0):
        print(f"Processing {model_name}")
        #skip model if already processed
        MODEL_NAME = model_name
        path_to_safe = f'/workspace/src/data/qrels_{modality}_pool_{MODEL_NAME.replace("/", "_")}.json'

        if os.path.exists(path_to_safe):
            print(f"Skipping {model_name} because it already exists")
            continue

        judge_osllm = OSLLMJudge( model_name=model_name, few_shot_count=0, num_gpus=1, device="cuda")
        all_judgments = single_process_topics(topics, pool_documents, judge_osllm)

        with open(path_to_safe, 'w', encoding='utf-8') as json_file:
            json.dump(all_judgments, json_file, ensure_ascii=False, indent=4)

        print("Auto qrels saved")

In [ ]:
for model_name in model_names:

    with torch.cuda.device(0):
        print(f"Processing {model_name}")
        #skip model if already processed
        MODEL_NAME = model_name
        path_to_safe = f'/workspace/src/data/qrels_{modality}_pool_{MODEL_NAME.replace("/", "_")}.json'

        if os.path.exists(path_to_safe):
            print(f"Skipping {model_name} because it already exists")
            continue

        judge_osllm = OSLLMJudge( model_name=model_name, few_shot_count=0, num_gpus=1, device="cuda")
        all_judgments = single_process_topics(topics, pool_documents, judge_osllm)

        with open(path_to_safe, 'w', encoding='utf-8') as json_file:
            json.dump(all_judgments, json_file, ensure_ascii=False, indent=4)

        print("Auto qrels saved")

In [ ]:
all_judgments